In [5]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

In [164]:
flow = InstalledAppFlow.from_client_secrets_file("dags/client_secrets.json", scopes=["https://www.googleapis.com/auth/youtube.readonly"])
    
credentials = flow.run_local_server(port=4040, authorization_prompt_message='')

youtube = build("youtube", "v3", credentials=credentials)

### My playlists

In [111]:
request = youtube.playlists().list(
        part="snippet,contentDetails",
        maxResults=50,
        mine=True
    )
response = request.execute()

In [112]:
playlists = []

In [113]:
for item in response['items']:
    if '💼' not in item['snippet']['title']: # remove non-music playlists
        playlists.append([item['id'], item['snippet']['title']])

In [163]:
len(playlists)

32

### Playlist videos

In [159]:
def get_playlist_videos(playlist_id: str, nextPageToken: str = ''):
    if nextPageToken:
        request = youtube.playlistItems().list(
                part="snippet,contentDetails",
                maxResults=50,
                playlistId=playlist_id,
                pageToken=nextPageToken
            )
    else:
        request = youtube.playlistItems().list(
                part="snippet,contentDetails",
                maxResults=50,
                playlistId=playlist_id
            )
    
    response = request.execute()
    return response

In [160]:
def populate_videos(response):
    for item in response['items']:
        if item['snippet']['title'] not in ('Deleted video', 'Private video') and item['contentDetails']['videoId'] not in videos:
            # songs.append([item['contentDetails']['videoId'], item['snippet']['title'], item['snippet']['videoOwnerChannelTitle']])
            videos[item['contentDetails']['videoId']] = (item['snippet']['title'], item['snippet']['videoOwnerChannelTitle'])

In [161]:
%%time
videos = {}
response = ''

for playlist in playlists:
    
    response = get_playlist_videos(playlist[0])
    populate_videos(response)

    while response.get('nextPageToken', ''):
        response = get_playlist_videos(playlist[0], response['nextPageToken'])
        populate_videos(response)

CPU times: total: 156 ms
Wall time: 10.7 s


In [162]:
len(videos)

724

### Save to BigQuery

### TODO: Extract from liked videos, extract from saved playlists

### TODO: Add duration if used in Spotify

In [42]:
request = youtube.videos().list(
        part="snippet,contentDetails,statistics",
        id="lXgF3WShEUo"
    )
response = request.execute()

In [46]:
response['items'][0]['contentDetails']['duration']

'PT3M20S'